In [1]:
import pandas as pd
import numpy as np
from huggingface_hub import InferenceClient

/Users/vincentfong/CS/cornell-admissions-faq-bot/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = InferenceClient(token="<key redacted>")

In [3]:
df = pd.read_json("faq_processed.json")

In [4]:
df["Embedded"] = df["Embedded"].apply(lambda x: client.feature_extraction(x, model="sentence-transformers/all-MiniLM-L6-v2"))

In [5]:
# Save cosine 
cosines = np.vstack(df["Embedded"])
np.save('embeddings.npy', cosines)
df.to_pickle("faq_with_embeddings.pkl")

In [6]:
df.head()

,Question,Answer,Source,Embedded,html
0,[Home > About Cornell > Academics] What is the...,Cornell undergraduates get to know their profe...,https://faq.enrollment.cornell.edu/kb/article/...,"[0.0762246, -0.1303518, 0.055723336, 0.0420286...","<div class=""hf-article_content"">\n <p>\n <spa..."
1,[Home > About Cornell > Academics] Can I doubl...,Cornell offers nearly 80 majors and more than ...,https://faq.enrollment.cornell.edu/kb/article/...,"[0.0745696, -0.0892729, 0.078019045, 0.0002153...","<div class=""hf-article_content"">\n <p>\n <spa..."
2,[Home > About Cornell > Academics] Does Cornel...,Several of the academic programs at Cornell of...,https://faq.enrollment.cornell.edu/kb/article/...,"[-0.028614562, -0.063003235, 0.052006, -0.0486...","<div class=""hf-article_content"">\n <p>\n <spa..."
3,[Home > About Cornell > Academics] What is Cor...,Cornell undergraduates can get to know their p...,https://faq.enrollment.cornell.edu/kb/article/...,"[0.04897193, -0.14860685, 0.0192861, -0.001755...","<div class=""hf-article_content"">\n <p>\n <spa..."
4,[Home > About Cornell > Academics] How do I ge...,Cornell University’s Courses of Study is publi...,https://faq.enrollment.cornell.edu/kb/article/...,"[0.031461887, -0.12976694, 0.012011488, 0.0492...","<div class=""hf-article_content"">\n <p>\n <spa..."


In [7]:
def semantic_search(query, cosines, df):
    query_embedding = client.feature_extraction(query, model="sentence-transformers/all-MiniLM-L6-v2")
    threshold = 0.5
    similarities = cosines@np.array(query_embedding)
    sims_above_threshold = np.sum(similarities > threshold)
    if sims_above_threshold > 1:
        match_idxs = np.argsort(similarities)[-2:][::-1]
        prompt = ", ".join(["[GENERATED ANSWER] " + df.loc[idx, "Answer"] for idx in match_idxs])
        messages = [
            {"role": "system", "content": "You are helping a retrieval system in providing correct answers to the FAQ section of Cornell University's general admissions page. You are given two possible answers, and you should choose and summarize the best one."},
            {"role": "user", "content": prompt}
        ]
    elif sims_above_threshold == 1:
        match_idx = np.argmax(similarities)
        prompt = df.loc[match_idx, "Answer"]
        messages = [
            {"role": "system", "content": "You are helping a retrieval system in providing correct answers to the FAQ section of Cornell University's general admissions page. You are given an answer that you should summarize."},
            {"role": "user", "content": prompt}
        ]
    else:
        messages = []

    completion = client.chat.completions.create(
        model="Qwen/Qwen2.5-72B-Instruct", 
        messages=messages
    )
    return {
        "Answer": completion.choices[0].message,
    }

In [10]:
user_query = "Where is Cornell University located?"
result = semantic_search(user_query, cosines, df)
print("Best Match:")
print(f"Answer: {result['Answer'].content}")

HfHubHTTPError: 500 Server Error: Internal Server Error for url: https://api-inference.huggingface.co/models/Qwen/Qwen2.5-72B-Instruct/v1/chat/completions (Request ID: 42IBMX)

Model too busy, unable to get response in less than 120 second(s)